In [66]:
import contextlib
import datetime
import os
import random
import time

import baostock as bs
import pandas as pd
from tqdm import tqdm

In [64]:
def supress_stdout(func):
    def wrapper(*a, **ka):
        with open(os.devnull, 'w') as devnull:
            with contextlib.redirect_stdout(devnull):
                return func(*a, **ka)
    return wrapper


@supress_stdout
def get_pe(code):
    """
    获取过去20天平均peTTM
    """
    start = (datetime.date.today() - datetime.timedelta(days=30)).isoformat()
    bs.login()
    rs = bs.query_history_k_data_plus(code, "date,code,close,peTTM", start, "", adjustflag="3")
    stock_pe = rs.get_data()
    bs.logout
    return round(stock_pe['peTTM'][-20:].astype('float64').mean(), 2)


def get_stock_list(hs300=True, zz500=True):
    """
    获取沪深300，中证500指数股票
    """
    stocks = []
    
    bs.login()
    
    if hs300:
        rs = bs.query_hs300_stocks()
        while (rs.error_code == '0') & rs.next():
            stocks.append(rs.get_row_data())
    
    if zz500:
        rs = bs.query_zz500_stocks()
        while (rs.error_code == '0') & rs.next():
            stocks.append(rs.get_row_data())            
        
    bs.logout()
    
    result = pd.DataFrame(stocks, columns=rs.fields)
    return result[['code', 'code_name']]

# # 输出个股历史的市盈率
# code = 'sz.002402'

# last_month = datetime.date.today().isoformat()[:7]
# key_date = ['1999-05', '2001-06', '2002-09', '2005-07', '2007-08', '2008-10', '2010-04', '2012-12', '2015-06', '2018-10',
#             '2019-01', '2020-01', '2021-01', last_month]

# start = (datetime.date.today() - datetime.timedelta(days=30)).isoformat()

# bs.login()
# rs = bs.query_history_k_data_plus(code, "date,code,close,peTTM", start, "", adjustflag="3")
# stock_pe = rs.get_data()
# bs.logout

# stock_pe_month = stock_pe
# stock_pe_month['month'] = stock_pe_month['date'].str.slice(0, 7)
# stock_pe_month['peTTM'] = pd.to_numeric(stock_pe_month['peTTM'])
# stock_month_stats = stock_pe_month.groupby(['month'])['peTTM'].agg(['mean'])

# df = pd.concat([stock_month_stats, pe_month_50, pe_month_75], axis=1).reindex(key_date).sort_index().dropna().rename(columns={"mean": code})
# display(df)

# fig = px.line(df, title='peTTM')
# fig.show()


In [98]:
stock_list = get_stock_list()
assert(len(stock_list) == 800)

# 去除银行
stock_list = stock_list[stock_list['code_name'].str.contains('银行') == False]
stock_list = stock_list[stock_list['code_name'].str.contains('商行') == False]
print("股票数", len(stock_list))

peTTM_list = []
for i in tqdm(range(len(stock_list))):
    code = stock_list.iloc[i]['code']
    peTTM = get_pe(code)
    peTTM_list.append(peTTM)
    # time.sleep(random.random() * 2)

stock_list.insert(2, "peTTM", peTTM_list, True)

stock_list = stock_list[stock_list['peTTM'] > 0]
stock_list = stock_list[stock_list['peTTM'] < 50]
stock_list = stock_list.sort_values(by=['peTTM'], ignore_index=True)
print("低市盈率股票数", len(stock_list))

stock_list.to_csv('stock_list.csv', index=False)

login success!
logout success!
股票数 764


100%|█████████████████████████████████████████████████████████████████████████| 764/764 [02:15<00:00,  5.63it/s]

低市盈率股票数 556


In [93]:
stock_list = stock_list[stock_list['peTTM'] > 0]

In [96]:
stock_list = stock_list.sort_values(by=['peTTM'], ignore_index=True)

In [97]:
stock_list.to_csv('stock_list.csv', index=False)